## 基础

全连接网络对图片的特征提取的问题（将一张图片上的所有像素点展开成一个1维向量输入网络）：

![20221128160037](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20221128160037.png)

在卷积神经网络中，计算范围是在像素点的空间邻域内进行的，卷积核参数的数目也远小于全连接层。卷积核本身与输入图片大小无关，它代表了对空间邻域内某种特征模式的提取。

卷积中，输入与输出的维度计算：(p为padding填充大小，k为kernel卷积核大小，s为stride步幅大小)

![20221128161140](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20221128161140.png)

比如，输入的图片大小为 28 * 28 ， kernel_size=5, stride=1, padding=2 ， 输出大小为 (28+2 * 2-5) / 1 +1 = 28 ， 与输入图片大小保持一致。


## 多通道、批量操作

多输入通道：

![20221128162327](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20221128162327.png)

多输出通道：

![20221128162432](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20221128162432.png)

批量操作：

![20221128162657](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20221128162657.png)


![20221128163552](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20221128163552.png)

## 池化、relu激活函数、Batch Normalization、Drop out

![20221129155413](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20221129155413.png)

![20221129161313](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20221129161313.png)

![20221129161125](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20221129161125.png)

![20221129161417](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20221129161417.png)

In [2]:
# 输入数据形状是 [N, K]时的示例

import numpy as np
import paddle
from paddle.nn import BatchNorm1D


data = np.array([[1,2,3], [4,5,6], [7,8,9]]).astype('float32')

# 使用BatchNorm1D计算归一化的输出
# 输入数据维度[N, K]，num_features等于K
bn = BatchNorm1D(num_features=3)    
x = paddle.to_tensor(data)
y = bn(x)
print('output of BatchNorm1D Layer: \n {}'.format(y.numpy()))

# 使用Numpy计算均值、方差和归一化的输出
# 这里对第0个特征进行验证
a = np.array([1,4,7])
a_mean = a.mean()
a_std = a.std()
b = (a - a_mean) / a_std
print('std {}, mean {}, \n output {}'.format(a_mean, a_std, b))

output of BatchNorm1D Layer: 
 [[-1.2247438 -1.2247438 -1.2247438]
 [ 0.         0.         0.       ]
 [ 1.2247438  1.2247438  1.2247438]]
std 4.0, mean 2.449489742783178, 
 output [-1.22474487  0.          1.22474487]


当输入数据形状是[N,C,H,W]时， 一般对应卷积层的输出。这种情况下`会沿着C这一维度进行展开`，分别`对每一个通道计算N个样本`中总共N×H×W个像素点的均值和方差

In [16]:
# 输入数据形状是[N, C, H, W]时的batchnorm示例

import numpy as np
import paddle
from paddle.nn import BatchNorm2D


np.random.seed(100)
data = np.random.rand(2,3,3,3).astype('float32')

# 使用BatchNorm2D计算归一化的输出
# 输入数据维度[N, C, H, W]，num_features等于C
bn = BatchNorm2D(num_features=3)
x = paddle.to_tensor(data)
y = bn(x)
print('input of BatchNorm2D Layer: \n {}'.format(x.numpy()))
print('output of BatchNorm2D Layer: \n {}'.format(y.numpy()))

# 取出data中第0通道的数据，
# 使用numpy计算均值、方差及归一化的输出
a = data[:, 0, :, :]
a_mean = a.mean()
a_std = a.std()
b = (a - a_mean) / a_std
print('channel 0 of input data: \n {}'.format(a))
print('std {}, mean {}, \n output: \n {}'.format(a_mean, a_std, b))

# 提示：这里通过numpy计算出来的输出
# 与BatchNorm2D算子的结果略有差别，
# 因为在BatchNorm2D算子为了保证数值的稳定性，
# 在分母里面加上了一个比较小的浮点数epsilon=1e-05

input of BatchNorm2D Layer: 
 [[[[0.54340494 0.2783694  0.4245176 ]
   [0.84477615 0.00471886 0.12156912]
   [0.67074907 0.82585275 0.13670659]]

  [[0.5750933  0.89132196 0.20920213]
   [0.18532822 0.10837689 0.21969749]
   [0.9786238  0.8116832  0.17194101]]

  [[0.81622475 0.27407375 0.4317042 ]
   [0.9400298  0.81764936 0.33611196]
   [0.17541045 0.37283206 0.00568851]]]


 [[[0.25242636 0.7956625  0.01525497]
   [0.5988434  0.6038045  0.10514768]
   [0.38194343 0.03647606 0.89041156]]

  [[0.98092085 0.05994199 0.89054596]
   [0.5769015  0.7424797  0.63018394]
   [0.5818422  0.02043913 0.21002658]]

  [[0.5446849  0.76911515 0.25069523]
   [0.2858957  0.8523951  0.9750065 ]
   [0.8848533  0.35950786 0.59885895]]]]
output of BatchNorm2D Layer: 
 [[[[ 0.4126078  -0.46198368  0.02029109]
   [ 1.4071034  -1.3650038  -0.97940934]
   [ 0.832831    1.344658   -0.9294571 ]]

  [[ 0.2520175   1.2038351  -0.84927964]
   [-0.9211378  -1.1527538  -0.8176896 ]
   [ 1.4666051   0.96413004 -0.96